#**Human Segmentation using YOLACT**
based on: https://github.com/dbolya/yolact　\
forked repo: https://github.com/yuma2ok/yolact_segmask


In [ ]:
#@markdown **NVIDIA GPU**
import subprocess
sub_p_res = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total,memory.free', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(sub_p_res)

In [ ]:
#@markdown **Setup Environment**\
#@markdown ライブラリのインストールなど


%cd /content/
# install requirements
!pip install cython
!pip install opencv-python pillow pycocotools matplotlib
!pip install torchvision==0.5.0
!pip install torch==1.4.0

# clone the repo
!git clone https://github.com/yuma2ok/yolact_segmask.git

# activate DCNv2
%cd /content/yolact_segmask/external/DCNv2
!python setup.py build develop

# get pretrained model
import gdown

!mkdir -p '/content/yolact_segmask/weights'
%cd /content/yolact_segmask/weights
!gdown 1ZPu1YR2UzGHQD0o1rEqy-j5bmEm3lbyP
weights = '/content/yolact_segmask/weights/yolact_plus_resnet50_54_800000.pth'

# make in-out img dir
inimgdir = '/content/yolact_segmask/in_img_files/'
!mkdir -p $inimgdir
outimgdir = '/content/yolact_segmask/out_img_files/'
!mkdir -p $outimgdir

# **Image 画像**


### **Setting**
セグメントしたい画像をフォルダ"in_img_files"に入れる

In [ ]:
# @markdown **入力画像を表示**

import os
import re
import matplotlib.pyplot as plt
import cv2

patternStr = '.+\.(jpg|jpeg|png)'
pattern = re.compile(patternStr)
for item in os.listdir(inimgdir):
    result = pattern.match(item)
    if result:
        file = os.path.join(inimgdir,item)
        print(file)
        img = cv2.cvtColor(cv2.imread(file), cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(10, 10))
        plt.imshow(img)
        plt.show()

### **Run**
eval.pyはフォーク元のレポジトリの実行ファイル\
eval_human.pyは人物のみの白黒マスク画像を返す実行ファイルです

In [ ]:
#　@markdown **Run eval.py**
%cd /content/yolact_segmask

!python eval.py --trained_model=weights/yolact_plus_resnet50_54_800000.pth\
                --score_threshold=0.15\
                --top_k=15\
                --display_bboxes=False\
                --display_text=False\
                --display_scores=False\
                --images=in_img_files/:out_img_files/

In [ ]:
#　@markdown **Run eval_human.py**\
# @markdown 白黒人物セグメンテーションを実行

%cd /content/yolact_segmask

!rm -rf $outimgdir
!mkdir -p $outimgdir

!python eval_human.py --trained_model=weights/yolact_plus_resnet50_54_800000.pth\
                --score_threshold=0.15\
                --top_k=15\
                --display_bboxes=False\
                --display_text=False\
                --display_scores=False\
                --images=in_img_files/:out_img_files/

In [ ]:
# @markdown **白黒反転**\
# @markdown 反転したい時のみ実行\
# @markdown 画像ファイルのパスを入力

import cv2
%cd /content/yolact_segmask/


filepath='/content/yolact_segmask/out_img_files/test.png' #@param {type:"string"}
img=cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
img=255-img
cv2.imwrite(filepath, img)

In [ ]:
# @markdown **出力画像を表示**

import os
import re
import matplotlib.pyplot as plt
import cv2

patternStr = '.+\.(jpg|jpeg|png)'
pattern = re.compile(patternStr)
for item in os.listdir(outimgdir):
    result = pattern.match(item)
    if result:
        file = os.path.join(outimgdir,item)
        print(file)
        img = cv2.cvtColor(cv2.imread(file), cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(10, 10))
        plt.imshow(img)
        plt.show()

# **Video 動画**
**処理の流れ**
1. 動画をフレームごとの静止画に変換
2. 50フレームごとに分割(エラー対策)
3. 全ての静止画のマスクを作成
4. 静止画を繋げて動画にする

### **Setting**
セグメントしたい動画をyolact_segmask内に入れる

In [ ]:
# @markdown **フレーム毎の静止画を作成**\
# @markdown vid_frame_filesへ出力される

import cv2
import os
%cd /content/yolact_segmask

videoimgdir = '/content/yolact_segmask/vid_frame_files/'
!rm -rf $videoimgdir
!mkdir -p $videoimgdir

def save_all_frames(video_path, dir_path, ext='jpg'):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        return

    digit = len(str(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))))

    n = 0

    while True:
        ret, frame = cap.read()
        if ret:
            cv2.imwrite('{}{}.{}'.format(videoimgdir, str(n).zfill(digit), ext), frame)
            n += 1
        else:
            return

input_video_path = '/content/yolact_segmask/seg.mp4' #@param {type:"string"}
save_all_frames(input_video_path, videoimgdir, 'png')

In [ ]:
# @markdown **vid_frame_files内の静止画を50枚ずつに分割**
import os
import shutil
frame_num = sum(os.path.isfile(os.path.join(videoimgdir, name)) for name in os.listdir(videoimgdir))
print("file num: {}".format(frame_num))
dir_num = frame_num // 50 + 1
print("dir num: {}".format(dir_num))
for i in range(1, dir_num+1):
  folder_path = videoimgdir + "_" + str(i*50)
  if(os.path.exists(folder_path)):
    shutil.rmtree(folder_path)
  os.mkdir(folder_path)
%cd /content/yolact_segmask/vid_frame_files
digit = len(str(frame_num))
cnt = 0
for i in range(frame_num):
  if(i % 50 == 0):
    cnt += 1
  filepath = "{}.png".format(str(i).zfill(digit))
  dirpath = videoimgdir + "_" + str(cnt*50) + "/" + filepath
  shutil.move(filepath, dirpath)

### **Run**
セグメンテーションの実行

In [ ]:
# @markdown **Run eval_human.py**\
# @markdown 人物の白黒セグメンテーションを実行\
# @markdown vid_frame_outへ出力される

%cd /content/yolact_segmask

vidframeout = "/content/yolact_segmask/vid_frame_out"

!rm -rf $vidframeout
!mkdir -p $vidframeout

for i in range(1, dir_num+1):
  folder_path = videoimgdir + "_" + str(i*50)
  !echo $i
  !python eval_human.py --trained_model=weights/yolact_plus_resnet50_54_800000.pth\
                --score_threshold=0.15\
                --top_k=15\
                --display_bboxes=False\
                --display_text=False\
                --display_scores=False\
                --images=$folder_path/:$vidframeout/

In [ ]:
# @markdown **vid_frame_outの白黒反転**\
# @markdown 反転したい時のみ実行
import cv2
%cd /content/yolact_segmask/vid_frame_out/
digit = len(str(frame_num))
for i in range(frame_num):
  filename = "{}.png".format(str(i).zfill(digit))
  img=cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
  img=255-img
  cv2.imwrite(filename,img)
  
print("done")

In [ ]:
# @markdown **出力画像を動画に変換**\
# @markdown segmented_videoへ出力される
vidoutdir = '/content/yolact_segmask/segmented_video/'
!rm -rf $vidoutdir
!mkdir -p $vidoutdir
vidpath = vidoutdir + 'seg_out.avi'

digit = len(str(frame_num))
if(digit == 1):
  imgpath = 'vid_frame_out/%01d.png'
elif(digit == 2):
  imgpath = 'vid_frame_out/%02d.png'
elif(digit == 3):
  imgpath = 'vid_frame_out/%03d.png'
else:
  imgpath = 'vid_frame_out/%04d.png'

%cd /content/yolact_segmask
!ffmpeg -f image2 -i $imgpath $vidpath
%cd $vidoutdir
!ffmpeg -i seg_out.avi seg_out.mp4

print("done")